## Generating Data Using PyMultiNest + Expressions

the output data is saved as 'F24s-.txt'

In [ ]:
%env DYLD_LIBRARY_PATH=/Users/vahid/libs/MultiNest/lib
%env LD_LIBRARY_PATH=/Users/vahid/libs/MultiNest/lib

In [ ]:
import numpy as np
import pymultinest
import pickle
from sympy import symbols, lambdify
import os

# ------------------------------------------------------
# Step 0: Ensure output directory exists
# ------------------------------------------------------
output_dir = 'mn_output'
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------
# Step 1: Load symbolic regression models (.pck files)
# ------------------------------------------------------
m0_sym, m12_sym, A0_sym, tanb_sym = symbols('m0 m12 A0 tanb')
input_vars = (m0_sym, m12_sym, A0_sym, tanb_sym)

def load_sympy_function(filename):
    with open(filename, 'rb') as f:
        expr = pickle.load(f)
    return lambdify(input_vars, expr, 'numpy')

compute_mh = load_sympy_function('./expressions/mH0-sympy.pck')
compute_gminus2 = load_sympy_function('./expressions/g-2-sympy.pck')
compute_omega = load_sympy_function('./expressions/Omega-sympy.pck')

# ------------------------------------------------------
# Step 2: Define priors and parameter space
# ------------------------------------------------------
param_names = ['m0', 'm12', 'A0', 'tanb']
param_bounds = {
    'm0': (0, 10000),       # 0 to 10 TeV in GeV
    'm12': (0, 10000),      # 0 to 10 TeV in GeV
    'A0': (-60000, 60000),  # -60 to 60 TeV in GeV
    'tanb': (1.5, 50)       # dimensionless
}

def prior_transform(cube, ndim, nparams):
    for i, key in enumerate(param_names):
        low, high = param_bounds[key]
        cube[i] = low + cube[i] * (high - low)

# ------------------------------------------------------
# Step 3: Gaussian Log-Likelihood based on benchmark values
# ------------------------------------------------------
# Benchmark observable values and uncertainties
mu_mh, sigma_mh = 125.1, 0.2
mu_g2, sigma_g2 = 2.51e-9, 1.0e-9
mu_omega, sigma_omega = 0.12, 0.01

# samples = []

def loglike(cube, ndim, nparams):
    m0, m12, A0, tanb = cube[0], cube[1], cube[2], cube[3]

    try:
        mh = compute_mh(m0, m12, A0, tanb)
        g2 = compute_gminus2(m0, m12, A0, tanb)
        omega = compute_omega(m0, m12, A0, tanb)
        cube[4], cube[5], cube[6] = mh , g2, omega 
        # Check for invalid results
        if not np.all(np.isfinite([mh, g2, omega])):
            return -1e90

        # Gaussian log-likelihood
        chi2 = ((mh - mu_mh) ** 2 / sigma_mh ** 2 +
                (g2 - mu_g2) ** 2 / sigma_g2 ** 2 +
                (omega - mu_omega) ** 2 / sigma_omega ** 2)

        # if chi2 > 1e6:  # reasonable cutoff
        #     return -1e90  # discard the sample
       
        logL = -0.5 * chi2

        # samples.append((m0, m12, A0, tanb, mh, g2, omega, logL))

        return logL

    except Exception:
        return -1e90

# ------------------------------------------------------
# Step 4: Run MultiNest
# ------------------------------------------------------
pymultinest.run(
    loglike,
    prior_transform,
    n_dims = 4,
    n_params= 7,
    n_live_points=2000,
    evidence_tolerance=0.5,
    sampling_efficiency='model',
    outputfiles_basename=f'{output_dir}/F24s-',
    resume=False,
    verbose=True,
    max_iter=20000
)

F24s = np.loadtxt('./mn_output/F24s-.txt')
print(f"\n✅ Saved {len(F24s)} samples to F24s-.txt with MultiNest weights.")


"""
Since we modified the code to save the observables in .txt file too,
There's no need for the code below
"""

# ------------------------------------------------------
# Step 5: Save output to F24-synthetic.txt
# ------------------------------------------------------
# Read MultiNest posterior samples
# posterior_file = f'{output_dir}/.txt'  # MultiNest's posterior file
# data = np.loadtxt(posterior_file)  # Columns: weight, logL, m0, m12, A0, tanb
# weights = data[:, 0]
# logLs = data[:, 1]
# params = data[:, 2:6]  # m0, m12, A0, tanb

# Compute observables for each sample
# observables = []
# for p in params:
#     m0, m12, A0, tanb = p
#     try:
#         mh = compute_mh(m0, m12, A0, tanb)
#         g2 = compute_gminus2(m0, m12, A0, tanb)
#         omega = compute_omega(m0, m12, A0, tanb)
#         if not np.all(np.isfinite([mh, g2, omega])):
#             mh, g2, omega = np.nan, np.nan, np.nan
#     except:
#         mh, g2, omega = np.nan, np.nan, np.nan
#     observables.append([mh, g2, omega])
# observables = np.array(observables)

# Combine weights, logL, parameters, and observables
# final_data = np.column_stack((weights, logLs, params, observables))
# np.savetxt('F24-synthetic.txt', final_data, fmt='%.18e')

# print(f"\n✅ Saved {len(final_data)} samples to F24-synthetic.txt with MultiNest weights.")


## Triangle plot of the F24s-

In [ ]:
import numpy as np
from getdist import MCSamples, plots

# Load data
samples = np.loadtxt("./mn_output/F24s-.txt")
psamples = samples[:, 2:]

names = ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']
axes = ['m0', 'm12', 'A0', 'tanb', 'gminus2', 'omega']
labels = [
    r'm_0', r'm_{1/2}', r'A_0', r'\tan\beta',
    r'M_{H^0}', r'\delta (g-2)_\mu', r'\Omega_{\rm DM}\, h^2'
]

# Create MCSamples
g_samp = MCSamples(samples=psamples,
                   names=names,
                   labels=labels,
                   name_tag="F24")


# Plot
g = plots.get_subplot_plotter(width_inch=8)
g.triangle_plot([g_samp], axes, plot_3d_with_param="mh", legend_labels=["F24s-"])

## Bayesflow (Likelihood-Free approach)

Using Bayesflow to generate new posterior samples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bayesflow
from bayesflow.summary_networks import DeepSet
from bayesflow.inference_networks import InvertibleNetwork
from bayesflow.amortizers import AmortizedPosterior
from bayesflow.trainers import Trainer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from scipy.stats import gaussian_kde
import os

# Load and preprocess data
data = np.loadtxt('./mn_output/F24s-.txt')
params = data[:, 2:6]         # 4 independent parameters: m0, m12, A0, tanb
observables = data[:, 6:9]    # 3 observables: mh, gminus2, omega

# Check for NaNs or infinite values
if np.any(np.isnan(data)) or np.any(np.isinf(data)):
    raise ValueError("Data contains NaNs or infinite values. Please clean the data.")

# Normalize data
def normalize(x):
    m, s = x.mean(0), x.std(0)
    s = np.where(s == 0, 1e-10, s)  # Avoid division by zero
    return (x - m) / s, m, s

p_norm, p_mean, p_std = normalize(params)
o_norm, o_mean, o_std = normalize(observables)

# Subsample for training and posterior sampling (one-fifth of data)
np.random.seed(42)
idx = np.random.choice(len(data), len(data)//5, replace=False)
params_sub, obs_sub = params[idx], observables[idx]
p_norm_sub, o_norm_sub = p_norm[idx], o_norm[idx]

# Train/val split
split = int(0.8 * len(p_norm_sub))
p_train, p_val = p_norm_sub[:split], p_norm_sub[split:]
o_train, o_val = o_norm_sub[:split], o_norm_sub[split:]

# Reshape to (N,1,D)
p3_train = p_train[:, None, :]
o3_train = o_train[:, None, :]
p3_val = p_val[:, None, :]
o3_val = o_val[:, None, :]

# Validate shapes
print(f"p3_train shape: {p3_train.shape}")
print(f"o3_train shape: {o3_train.shape}")
if p3_train.shape[0] != o3_train.shape[0]:
    raise ValueError("Mismatch in number of training samples between p3_train and o3_train.")
if p3_train.shape[0] == 0:
    raise ValueError("Training data is empty.")

sim_dict = {'prior_draws': p3_train, 'sim_data': o3_train}

# Define BayesFlow networks
summary_net = DeepSet(
    summary_dim=8,
    num_dense_s1=2,
    num_dense_s2=2,
    num_dense_s3=1,
    dense_s1_args={"units": 64, "activation": "relu"},
    dense_s2_args={"units": 64, "activation": "relu"},
    dense_s3_args={"units": 64, "activation": "relu"},
    pooling_fun="mean"
)

inference_net = InvertibleNetwork(
    num_params=4,
    num_coupling_layers=5,
    use_act_norm=True
)

amortizer = AmortizedPosterior(
    summary_net=summary_net,
    inference_net=inference_net
)

# Initialize and train the model with smaller batch size
trainer = Trainer(amortizer=amortizer, generative_model=None)
trainer.train_offline(sim_dict, epochs=50, batch_size=64,  # Reduced from 128
                     callbacks=[EarlyStopping(patience=10), ReduceLROnPlateau()])

# Generate posterior samples for the SUBSAMPLED dataset
o_sub = o_norm_sub[:, None, :]  # Reshape subsampled observables
input_dict = {'summary_conditions': o_sub}
post_samples = amortizer.sample(input_dict, n_samples=100)  # Generate 100 samples per observable
post_denorm = post_samples * p_std[None, None, :] + p_mean[None, None, :]  # Denormalize

# Save
np.savetxt("BFLF-post.txt", post_denorm.reshape(-1, 4), header="m0 m12 A0 tanb", fmt="%.6f")

print("Posterior samples saved to BFLF-post.txt")

## Bayesflow (Likelihood-Based)

In [ ]:

# Simulator
def simulator(theta, mu_mh=125.0, sigma_mh=0.5, mu_g2=0.0, sigma_g2=1e-10, mu_omega=0.12, sigma_omega=0.01):
    m0, m12, A0, tanb = theta[:, 0], theta[:, 1], theta[:, 2], theta[:, 3]
    try:
        mh = np.array([compute_mh(m0[i], m12[i], A0[i], tanb[i]) for i in range(len(m0))])
        g2 = np.array([compute_gminus2(m0[i], m12[i], A0[i], tanb[i]) for i in range(len(m0))])
        omega = np.array([compute_omega(m0[i], m12[i], A0[i], tanb[i]) for i in range(len(m0))])

        mh += np.random.normal(0, sigma_mh, size=mh.shape)
        g2 += np.random.normal(0, sigma_g2, size=g2.shape)
        omega += np.random.normal(0, sigma_omega, size=omega.shape)

        if not np.all(np.isfinite([mh, g2, omega])):
            mh, g2, omega = np.zeros_like(mh), np.zeros_like(g2), np.zeros_like(omega)

        sim2d = np.stack([mh, g2, omega], axis=-1)
        return sim2d
    except Exception:
        return np.zeros((len(m0), 3))

# Prior
def prior(batch_size):
    m0 = np.random.uniform(0, 10000, batch_size)
    m12 = np.random.uniform(0, 10000, batch_size)
    A0 = np.random.uniform(-60000, 60000, batch_size)
    tanb = np.random.uniform(1.5, 50, batch_size)
    return np.stack([m0, m12, A0, tanb], axis=-1)


# Offline data dictionary
sim_dict = {
    'prior_draws': p3_train,
    'sim_data': s3_train,
}

# Define networks
summary_net = DeepSet(
    summary_dim=8,
    num_dense_s1=2, num_dense_s2=2, num_dense_s3=2,
    dense_s1_args={"units": 64, "activation": "relu"},
    dense_s2_args={"units": 64, "activation": "relu"},
    dense_s3_args={"units": 64, "activation": "relu"},
    pooling_fun="mean"
)

inference_net = InvertibleNetwork(
    num_params=4,
    num_coupling_layers=5,
    use_act_norm=True
)
generative_model = GenerativeModel(prior, simulator, simulator_is_batched=True, prior_is_batched=True)
amortizer = AmortizedPosterior(summary_net=summary_net, inference_net=inference_net)

trainer = Trainer(amortizer=amortizer, generative_model=generative_model)

# Train
trainer.train_offline(sim_dict, epochs=39, batch_size=64,
                      callbacks=[EarlyStopping(patience=10), ReduceLROnPlateau()])

# Generate posterior samples
post_samples = amortizer.sample(input_dict, n_samples=100)



## Plotting Bayesflow vs MultiNest
Since BayesFlow is designed to infer the posterior distribution of the parameters <math xmlns="http://www.w3.org/1998/Math/MathML"><semantics><mrow><mi>p</mi><mo stretchy="false">(</mo><mi>m</mi><mn>0</mn><mo separator="true">,</mo><mi>m</mi><mn>12</mn><mo separator="true">,</mo><mi>A</mi><mn>0</mn><mo separator="true">,</mo><mi>t</mi><mi>a</mi><mi>n</mi><mi>b</mi><mi mathvariant="normal">∣</mi><mi>m</mi><mi>h</mi><mo separator="true">,</mo><mi>g</mi><mi>m</mi><mi>i</mi><mi>n</mi><mi>u</mi><mi>s</mi><mn>2</mn><mo separator="true">,</mo><mi>o</mi><mi>m</mi><mi>e</mi><mi>g</mi><mi>a</mi><mo stretchy="false">)</mo></mrow><annotation encoding="application/x-tex"> p(m0, m12, A0, tanb | mh, gminus2, omega) </annotation></semantics></math>, using the observables as conditioning data.



In [ ]:
import numpy as np
from getdist import MCSamples, plots

# Load BayesFlow posteriors
bf_pst = np.loadtxt('BFLF-post.txt', skiprows=1) 

# Load MultiNest data
data = np.loadtxt('./mn_output/F24s-.txt')
params = data[:, 2:6]  
weights = data[:, 0]  

# Define parameter names and labels
names = ['m0', 'm12', 'A0', 'tanb']
labels = [r'm_0', r'm_{12}', r'A_0', r'\tan\beta']

# Create MCSamples objects
bflf_samples = MCSamples(samples=bf_pst, names=names, labels=labels, 
                       label='BayesFlow Posteriors')
orig_samples = MCSamples(samples=params, weights=weights, names=names, 
                         labels=labels, label='MultiNest Posteriors')

# Plot triangle plot
g = plots.get_subplot_plotter()
g.triangle_plot([bflf_samples, orig_samples], ['m0', 'm12', 'A0', 'tanb'], 
                filled=True, colors=['blue', 'red'], 
                legend_labels=['BayesFlow-LF', 'MultiNest'])

print(bflf_samples.getTable().tableTex())
print(orig_samples.getTable().tablePNG())

In [ ]:
# print(bflf_samples.getTable().tableTex())

# print(orig_samples.getTable().tableTex())

***BayesFlow-LikelihoodFree***
| Parameter        | 95% limits          |
|------------------|----------------------|
| $m_0$            | $3698^{+5000}_{-4000}$      |
| $m_{12}$         | $6308^{+3000}_{-4000}$      |
| $A_0$            | $32600^{+30000}_{-30000}$   |
| $\tan\beta$      | $29^{+20}_{-20}$            |

***MultiNest***

| Parameter        | 95% limits          |
|------------------|----------------------|
| $m_0$            | $3542^{+6000}_{-3000}$      |
| $m_{12}$         | $6189^{+3000}_{-3000}$      |
| $A_0$            | $32000^{+30000}_{-30000}$   |
| $\tan\beta$      | $28^{+20}_{-20}$            |

In [ ]:
# Plot each parameter 1D marginalized posterior with normalized=True
for param in names:
    # Create a GetDist plotter instance
    g = plots.get_single_plotter(width_inch=4)
    g.plot_1d([bflf_samples, orig_samples], param, marker=0, title_limit=1, normalized=True, colors=['blue', 'red'])

## Harmonic

### RealNVPModel

In [ ]:
import numpy as np
import harmonic as hm

# Section 1: Load MultiNest Output
# -------------------------------
# Load the data file containing posterior weights, log-likelihoods, and parameters
# Column 1: posterior weights
# Column 2: log-likelihoods
# Columns 3-9: parameters ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']

def load_data(file_path):
    """Load the data from the specified file and check for numerical issues."""
    data = np.loadtxt(file_path)
    weights = data[:, 0]  # Posterior weights
    log_likelihoods = data[:, 1]  # Log-likelihoods (already in logL form)
    samples = data[:, 2:]  # Parameters (7 dimensions)
    
    # Check for inf/nan in log-likelihoods and samples
    if np.any(np.isinf(log_likelihoods)) or np.any(np.isnan(log_likelihoods)):
        print("Warning: log_likelihoods contain inf/nan values")
    if np.any(np.isinf(samples)) or np.any(np.isnan(samples)):
        print("Warning: samples contain inf/nan values")
    
    # Clip log-likelihoods to prevent extreme values
    log_likelihoods = np.clip(log_likelihoods, -1e10, 1e10)
    
    # Print data statistics for debugging
    print(f"Number of samples: {len(samples)}")
    print(f"Log-likelihood range: {log_likelihoods.min():.4f} to {log_likelihoods.max():.4f}")
    
    return weights, log_likelihoods, samples

file_path = './mn_output/F24s-.txt'
weights, log_likelihoods, samples = load_data(file_path)

# Section 2: Prepare Data for Harmonic
# -----------------------------------
# Reshape samples and log-likelihoods into pseudo-chains for harmonic
n_samples_total = samples.shape[0]  # Total number of samples
n_dim = samples.shape[1]  # 7 parameters
n_chains = 4  # Increased to 4 chains for better error estimation
n_samples_per_chain = n_samples_total // n_chains  # Integer division for equal splits

# Check if enough samples per chain
if n_samples_per_chain < 200:
    print(f"Warning: Only {n_samples_per_chain} samples per chain. Consider increasing samples.")

# Trim samples to ensure they fit n_chains * n_samples_per_chain
samples = samples[:n_chains * n_samples_per_chain]
log_weights = np.log(weights[:n_chains * n_samples_per_chain])
log_weights -= np.max(log_weights)

# Reshape to (n_chains, n_samples_per_chain, n_dim)
samples = samples.reshape(n_chains, n_samples_per_chain, n_dim)
log_weights = log_weights.reshape(n_chains, n_samples_per_chain)

# Initialize Chains object
chains = hm.Chains(ndim=n_dim)

# Add samples and log-likelihoods to Chains
chains.add_chains_3d(samples, ln_posterior=log_weights)

# Split data into training and inference sets with more inference samples
chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.3)  # Reduced to 30% training

# Section 3: Train Normalizing Flow Model
# --------------------------------------
# Initialize and train a RealNVPModel for evidence estimation
model = hm.model.RealNVPModel(
    ndim_in=n_dim,
    n_scaled_layers=1,          # Further reduced for stability
    n_unscaled_layers=2,        # Further reduced for stability
    learning_rate=0.001,        # Kept for convergence
    standardize=True,           # Standardize data
    temperature=1.0,            # Increased to 1.0 for smoother distribution
)

# Train model and print loss for debugging
model.fit(chains_train.samples)

# Section 4: Compute Evidence
# ---------------------------
# Use harmonic to compute the evidence and its error with clipping
ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv

# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = max(abs(err_ln_inv_evidence[0]), abs(err_ln_inv_evidence[1]))

# Section 5: Display Results
# -------------------------
# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RealNVPModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")


# Parse /data-synthetic/stats.dat to get Global Log-Evidence from synthetic
with open('./mn_output/F24s-stats.dat', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if 'Nested Sampling Global Log-Evidence' in line:
            # Extract the number after ':'
            mn_logZ = float(line.split(':')[1].split('+/-')[0].strip())
            err_mn_logZ = float(line.split(':')[1].split('+/-')[1].strip())
            break

print(f"PyMultiNest log evidence (ln(Z)): {mn_logZ:.4f} +/- {err_mn_logZ:.4f}")

### RQSplineModel

In [ ]:
import numpy as np
import harmonic as hm

# Section 1: Load MultiNest Output
# -------------------------------
# Load the data file containing posterior weights, log-likelihoods, and parameters
# Column 1: posterior weights
# Column 2: log-likelihoods
# Columns 3-9: parameters ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']

def load_data(file_path):
    """Load the data from the specified file and check for numerical issues."""
    data = np.loadtxt(file_path)
    weights = data[:, 0]  # Posterior weights
    log_likelihoods = data[:, 1]  # Log-likelihoods (already in logL form)
    samples = data[:, 2:]  # Parameters (7 dimensions)
    
    # Check for inf/nan in log-likelihoods and samples
    if np.any(np.isinf(log_likelihoods)) or np.any(np.isnan(log_likelihoods)):
        print("Warning: log_likelihoods contain inf/nan values")
    if np.any(np.isinf(samples)) or np.any(np.isnan(samples)):
        print("Warning: samples contain inf/nan values")
    
    # Clip log-likelihoods to prevent extreme values
    log_likelihoods = np.clip(log_likelihoods, -1e10, 1e10)
    
    # Print data statistics for debugging
    print(f"Number of samples: {len(samples)}")
    print(f"Log-likelihood range: {log_likelihoods.min():.4f} to {log_likelihoods.max():.4f}")
    
    return weights, log_likelihoods, samples

file_path = './mn_output/F24s-.txt'
weights, log_likelihoods, samples = load_data(file_path)

# Section 2: Prepare Data for Harmonic
# -----------------------------------
# Reshape samples and log-likelihoods into pseudo-chains for harmonic
n_samples_total = samples.shape[0]  # Total number of samples
n_dim = samples.shape[1]  # 7 parameters
n_chains = 8  # Increased to 8 to allow training_proportion=0.25
n_samples_per_chain = n_samples_total // n_chains  # Integer division for equal splits

# Check if enough samples per chain
if n_samples_per_chain < 200:
    print(f"Warning: Only {n_samples_per_chain} samples per chain. Consider increasing samples.")

# Trim samples to ensure they fit n_chains * n_samples_per_chain
samples = samples[:n_chains * n_samples_per_chain]
log_weights = np.log(weights[:n_chains * n_samples_per_chain])
log_weights -= np.max(log_weights)

# Reshape to (n_chains, n_samples_per_chain, n_dim)
samples = samples.reshape(n_chains, n_samples_per_chain, n_dim)
log_weights = log_weights.reshape(n_chains, n_samples_per_chain)


# Initialize Chains object
chains = hm.Chains(ndim=n_dim)

# Add samples and log of posterior weights to Chains
chains.add_chains_3d(samples, ln_posterior=log_weights)

# Split data into training and inference sets with more inference samples
chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.25)  # 25% training

# Print inference sample count for debugging
print(f"Inference samples: {chains_infer.samples.shape[0] * chains_infer.samples.shape[1]}")

# Section 3: Train Normalizing Flow Model
# --------------------------------------
# Initialize and train an RQSplineModel for evidence estimation
model = hm.model.RQSplineModel(
    ndim_in=n_dim,
    n_layers=6,                 # Reduced from default 8 for stability
    n_bins=8,                   # Default, kept for flexibility
    hidden_size=[128, 128],       # Default, kept for capacity
    spline_range=(-10, 10),   # Narrowed to constrain transformations
    standardize=True,           # Standardize data
    learning_rate=0.001,        # Kept for stable convergence
    momentum=0.9,               # Default for Adam optimizer
    temperature=0.5             # Kept to tighten distribution
)

# Train model
model.fit(chains_train.samples)

# Section 4: Compute Evidence
# ---------------------------
# Use harmonic to compute the evidence and its error
ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv

# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = max(abs(err_ln_inv_evidence[0]), abs(err_ln_inv_evidence[1]))

# Section 5: Display Results
# -------------------------
# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RQSplineModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")

print(f"PyMultiNest log evidence (ln(Z)): {mn_logZ:.4f} +/- {err_mn_logZ:.4f}")

## Random Scan over Parameter Space

### P_r = 1 , LogL = 0

In [ ]:
import numpy as np
import pickle
import os
from sympy import symbols, lambdify
from tqdm import tqdm

# ------------------------------------------------------
# Step 0: Ensure output directory exists
# ------------------------------------------------------
output_dir = 'randomscan_output'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'F24s_random.txt')

# ------------------------------------------------------
# Step 1: Load symbolic regression models
# ------------------------------------------------------
m0_sym, m12_sym, A0_sym, tanb_sym = symbols('m0 m12 A0 tanb')
input_vars = (m0_sym, m12_sym, A0_sym, tanb_sym)

def load_sympy_function(filename):
    with open(filename, 'rb') as f:
        expr = pickle.load(f)
    return lambdify(input_vars, expr, 'numpy')

compute_mh = load_sympy_function('./expressions/mH0-sympy.pck')
compute_g2 = load_sympy_function('./expressions/g-2-sympy.pck')
compute_omega = load_sympy_function('./expressions/Omega-sympy.pck')

# ------------------------------------------------------
# Step 2: Parameter bounds
# ------------------------------------------------------
param_bounds = {
    'm0': (0, 10000),
    'm12': (0, 10000),
    'A0': (-60000, 60000),
    'tanb': (1.5, 50)
}
param_names = list(param_bounds.keys())

# ------------------------------------------------------
# Step 3: Perform random scan
# ------------------------------------------------------
n_samples = 1000000
data = []

for _ in tqdm(range(n_samples), desc="Random scanning"):
    # Sample random parameters from uniform prior
    m0 = np.random.uniform(*param_bounds['m0'])
    m12 = np.random.uniform(*param_bounds['m12'])
    A0 = np.random.uniform(*param_bounds['A0'])
    tanb = np.random.uniform(*param_bounds['tanb'])

    try:
        # Compute observables
        mh = compute_mh(m0, m12, A0, tanb)
        g2 = compute_g2(m0, m12, A0, tanb)
        omega = compute_omega(m0, m12, A0, tanb)

        if not np.all(np.isfinite([mh, g2, omega])):
            continue  # skip invalid samples

        # Format: posterior_weight | log_likelihood | m0 m12 A0 tanb | mh g2 omega
        row = [1.0, 0.0, m0, m12, A0, tanb, mh, g2, omega]
        data.append(row)

    except Exception:
        continue  # skip any errors silently

# ------------------------------------------------------
# Step 4: Save output file
# ------------------------------------------------------
data = np.array(data)
np.savetxt(output_file, data, fmt='%.6e')
print(f"\n✅ Saved {len(data)} samples to {output_file}")


### P_r = 1, LogL != 0

In [ ]:
import numpy as np
import pickle
import os
from sympy import symbols, lambdify
from tqdm import tqdm

# ------------------------------------------------------
# Step 0: Ensure output directory exists
# ------------------------------------------------------
output_dir = 'randomscan_output'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'F24s_random_withlogL.txt')

# ------------------------------------------------------
# Step 1: Load symbolic regression models
# ------------------------------------------------------
m0_sym, m12_sym, A0_sym, tanb_sym = symbols('m0 m12 A0 tanb')
input_vars = (m0_sym, m12_sym, A0_sym, tanb_sym)

def load_sympy_function(filename):
    with open(filename, 'rb') as f:
        expr = pickle.load(f)
    return lambdify(input_vars, expr, 'numpy')

compute_mh = load_sympy_function('./expressions/mH0-sympy.pck')
compute_g2 = load_sympy_function('./expressions/g-2-sympy.pck')
compute_omega = load_sympy_function('./expressions/Omega-sympy.pck')

# ------------------------------------------------------
# Step 2: Parameter bounds
# ------------------------------------------------------
param_bounds = {
    'm0': (0, 10000),
    'm12': (0, 10000),
    'A0': (-60000, 60000),
    'tanb': (1.5, 50)
}

# ------------------------------------------------------
# Step 3: Observational values for likelihood
# ------------------------------------------------------
# These are placeholders; replace with correct experimental values and errors.
mh_obs, mh_err = 125.1, 2.0
g2_obs, g2_err = 2.5e-9, 0.5e-9
omega_obs, omega_err = 0.12, 0.01

def log_likelihood(mh, g2, omega):
    chi2_mh = ((mh - mh_obs) / mh_err) ** 2
    chi2_g2 = ((g2 - g2_obs) / g2_err) ** 2
    chi2_omega = ((omega - omega_obs) / omega_err) ** 2
    chi2_total = chi2_mh + chi2_g2 + chi2_omega
    return -0.5 * chi2_total

# ------------------------------------------------------
# Step 4: Perform random scan
# ------------------------------------------------------
n_samples = 1000000
data = []

for _ in tqdm(range(n_samples), desc="Random scanning"):
    m0 = np.random.uniform(*param_bounds['m0'])
    m12 = np.random.uniform(*param_bounds['m12'])
    A0 = np.random.uniform(*param_bounds['A0'])
    tanb = np.random.uniform(*param_bounds['tanb'])

    try:
        mh = compute_mh(m0, m12, A0, tanb)
        g2 = compute_g2(m0, m12, A0, tanb)
        omega = compute_omega(m0, m12, A0, tanb)

        if not np.all(np.isfinite([mh, g2, omega])):
            continue

        logL = log_likelihood(mh, g2, omega)
        row = [1.0, logL, m0, m12, A0, tanb, mh, g2, omega]
        data.append(row)

    except Exception:
        continue

# ------------------------------------------------------
# Step 5: Save output file
# ------------------------------------------------------
data = np.array(data)
np.savetxt(output_file, data, fmt='%.6e')
print(f"\n✅ Saved {len(data)} samples with log-likelihoods to {output_file}")


## Harmonic Evidence on Random Scans

### P_r = 1, LogL = 0

In [ ]:
import numpy as np
import harmonic as hm

# Section 1: Load Random Scan Output
# -------------------------------
# Load the data file containing posterior weights, log-likelihoods, and parameters
# Column 1: posterior weights
# Column 2: log-likelihoods
# Columns 3-9: parameters ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']

def load_data(file_path):
    """Load the data from the specified file and check for numerical issues."""
    data = np.loadtxt(file_path)
    weights = data[:, 0]  # Posterior weights
    log_likelihoods = data[:, 1]  # Log-likelihoods (already in logL form)
    samples = data[:, 2:]  # Parameters (7 dimensions)
    
    # Check for inf/nan in log-likelihoods and samples
    if np.any(np.isinf(log_likelihoods)) or np.any(np.isnan(log_likelihoods)):
        print("Warning: log_likelihoods contain inf/nan values")
    if np.any(np.isinf(samples)) or np.any(np.isnan(samples)):
        print("Warning: samples contain inf/nan values")
    
    # Clip log-likelihoods to prevent extreme values
    log_likelihoods = np.clip(log_likelihoods, -1e10, 1e10)
    
    # Print data statistics for debugging
    print(f"Number of samples: {len(samples)}")
    print(f"Log-likelihood range: {log_likelihoods.min():.4f} to {log_likelihoods.max():.4f}")
    
    return weights, log_likelihoods, samples

file_path = './randomscan_output/F24s_random.txt'
weights, log_likelihoods, samples = load_data(file_path)

# Reshape to (n_chains, n_samples_per_chain, n_dim)
n_samples_total = samples.shape[0]
n_chains = 4
n_samples_per_chain = n_samples_total // n_chains

# Trim samples to ensure they fit n_chains * n_samples_per_chain
samples = samples[:n_chains * n_samples_per_chain]
log_weights = np.log(weights[:n_chains * n_samples_per_chain])
log_weights -= np.max(log_weights)

# Reshape to (n_chains, n_samples_per_chain, n_dim)
samples = samples.reshape(n_chains, n_samples_per_chain, n_dim)
log_weights = log_weights.reshape(n_chains, n_samples_per_chain)

chains = hm.Chains(ndim=n_dim)
chains.add_chains_3d(samples, ln_posterior=log_weights)

chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.7)

model = hm.model.RealNVPModel(
    ndim_in=n_dim,
    n_scaled_layers=4,
    n_unscaled_layers=8,
    learning_rate=0.001,
    standardize=True,
    temperature=1.0,
)

model.fit(chains_train.samples)

ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv

# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = max(abs(err_ln_inv_evidence[0]), abs(err_ln_inv_evidence[1]))

# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RealNVPModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")


In [ ]:
model = hm.model.RQSplineModel(
    ndim_in=n_dim,
    n_layers=4,                 # Reduced from default 8 for stability
    n_bins=8,                   # Default, kept for flexibility
    hidden_size=[64, 64],       # Default, kept for capacity
    spline_range=(-5.0, 5.0),   # Narrowed to constrain transformations
    standardize=True,           # Standardize data
    learning_rate=0.001,        # Kept for stable convergence
    momentum=0.9,               # Default for Adam optimizer
    temperature=0.5             # Kept to tighten distribution
)

model.fit(chains_train.samples)

ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv

# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = max(abs(err_ln_inv_evidence[0]), abs(err_ln_inv_evidence[1]))

# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RQSplineModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")

### P_r = 1, LogL != 0

In [ ]:
import numpy as np
import harmonic as hm

# Section 1: Load Random Scan Output
# -------------------------------
# Load the data file containing posterior weights, log-likelihoods, and parameters
# Column 1: posterior weights
# Column 2: log-likelihoods
# Columns 3-9: parameters ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']

def load_data(file_path):
    """Load the data from the specified file and check for numerical issues."""
    data = np.loadtxt(file_path)
    weights = data[:, 0]  # Posterior weights
    log_likelihoods = data[:, 1]  # Log-likelihoods (already in logL form)
    samples = data[:, 2:]  # Parameters (7 dimensions)
    
    # Check for inf/nan in log-likelihoods and samples
    if np.any(np.isinf(log_likelihoods)) or np.any(np.isnan(log_likelihoods)):
        print("Warning: log_likelihoods contain inf/nan values")
    if np.any(np.isinf(samples)) or np.any(np.isnan(samples)):
        print("Warning: samples contain inf/nan values")
    
    # Clip log-likelihoods to prevent extreme values
    log_likelihoods = np.clip(log_likelihoods, -1e10, 1e10)
    
    # Print data statistics for debugging
    print(f"Number of samples: {len(samples)}")
    print(f"Log-likelihood range: {log_likelihoods.min():.4f} to {log_likelihoods.max():.4f}")
    
    return weights, log_likelihoods, samples

file_path = './randomscan_output/F24s_random_withlogL.txt'
weights, log_likelihoods, samples = load_data(file_path)

# Normalize and threshold log-likelihoods
log_likelihoods -= np.max(log_likelihoods)  # Shift max to 0
logL_threshold = -25  # Cutoff to reduce dynamic range
valid = log_likelihoods > logL_threshold
samples = samples[valid]
log_likelihoods = log_likelihoods[valid]

# Reshape to (n_chains, n_samples_per_chain, n_dim)
n_samples_total = samples.shape[0]
n_chains = 4
n_samples_per_chain = n_samples_total // n_chains

samples = samples[:n_chains * n_samples_per_chain]
log_likelihoods = log_likelihoods[:n_chains * n_samples_per_chain]

samples = samples.reshape(n_chains, n_samples_per_chain, n_dim)
log_likelihoods = log_likelihoods.reshape(n_chains, n_samples_per_chain)

chains = hm.Chains(ndim=n_dim)
chains.add_chains_3d(samples, ln_posterior=log_likelihoods)

chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.7)

model = hm.model.RealNVPModel(
    ndim_in=n_dim,
    n_scaled_layers=4,
    n_unscaled_layers=8,
    learning_rate=0.001,
    standardize=True,
    temperature=1.0,
)

model.fit(chains_train.samples)

ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv

# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract first error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = err_ln_inv_evidence[0] if not (np.isinf(err_ln_inv_evidence[0]) or np.isnan(err_ln_inv_evidence[0])) else 1.0  # Default to 1.0 if inf/nan


# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RealNVPModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")

# Parse /data-synthetic/stats.dat to get Global Log-Evidence from synthetic
with open('./mn_output/F24s-stats.dat', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if 'Nested Sampling Global Log-Evidence' in line:
            # Extract the number after ':'
            mn_logZ = float(line.split(':')[1].split('+/-')[0].strip())
            err_mn_logZ = float(line.split(':')[1].split('+/-')[1].strip())
            break

print(f"PyMultiNest log evidence (ln(Z)): {mn_logZ:.4f} +/- {err_mn_logZ:.4f}")

**Case**: `chains.add_chains_3d(samples, ln_posterior=log_weights)`
```
ln_inv_evidence: -54.8624
err_ln_inv_evidence: (-0.004039131973917959, 0.004022882996576763)
Harmonic Log Evidence (RealNVPModel): 54.8624 +/- -0.0040
PyMultiNest log evidence (ln(Z)): -10.8605 +/- 0.0583
```

In [ ]:
import numpy as np
import harmonic as hm

# Section 1: Load randomscan
# -------------------------------
# Load the data file containing posterior weights, log-likelihoods, and parameters
# Column 1: posterior weights
# Column 2: log-likelihoods
# Columns 3-9: parameters ['m0', 'm12', 'A0', 'tanb', 'mh', 'gminus2', 'omega']

def load_data(file_path):
    """Load the data from the specified file and check for numerical issues."""
    data = np.loadtxt(file_path)
    weights = data[:, 0]  # Posterior weights
    log_likelihoods = data[:, 1]  # Log-likelihoods (already in logL form)
    samples = data[:, 2:]  # Parameters (7 dimensions)
    
    # Check for inf/nan in log-likelihoods and samples
    if np.any(np.isinf(log_likelihoods)) or np.any(np.isnan(log_likelihoods)):
        print("Warning: log_likelihoods contain inf/nan values")
    if np.any(np.isinf(samples)) or np.any(np.isnan(samples)):
        print("Warning: samples contain inf/nan values")
    
    # Clip log-likelihoods to prevent extreme values
    log_likelihoods = np.clip(log_likelihoods, -1e10, 1e10)
    
    # Print data statistics for debugging
    print(f"Number of samples: {len(samples)}")
    print(f"Log-likelihood range: {log_likelihoods.min():.4f} to {log_likelihoods.max():.4f}")
    
    return weights, log_likelihoods, samples

file_path = './randomscan_output/F24s_random_withlogL.txt'
weights, log_likelihoods, samples = load_data(file_path)

# Normalize and threshold log-likelihoods
log_likelihoods -= np.max(log_likelihoods)  # Shift max to 0
logL_threshold = -25  # Cutoff to reduce dynamic range
valid = log_likelihoods > logL_threshold
samples = samples[valid]
log_likelihoods = log_likelihoods[valid]

# Reshape to (n_chains, n_samples_per_chain, n_dim)
n_samples_total = samples.shape[0]
n_chains = 4
n_samples_per_chain = n_samples_total // n_chains

samples = samples[:n_chains * n_samples_per_chain]
log_likelihoods = log_likelihoods[:n_chains * n_samples_per_chain]

samples = samples.reshape(n_chains, n_samples_per_chain, n_dim)
log_likelihoods = log_likelihoods.reshape(n_chains, n_samples_per_chain)

chains = hm.Chains(ndim=n_dim)
chains.add_chains_3d(samples, ln_posterior=log_likelihoods)

chains_train, chains_infer = hm.utils.split_data(chains, training_proportion=0.7)

model = hm.model.RQSplineModel(
    ndim_in=n_dim,
    n_layers=4,                 # Reduced from default 8 for stability
    n_bins=8,                   # Default, kept for flexibility
    hidden_size=[64, 64],       # Default, kept for capacity
    spline_range=(-5.0, 5.0),   # Narrowed to constrain transformations
    standardize=True,           # Standardize data
    learning_rate=0.001,        # Kept for stable convergence
    momentum=0.9,               # Default for Adam optimizer
    temperature=0.5             # Kept to tighten distribution
)

model.fit(chains_train.samples)

ev = hm.Evidence(chains_infer.nchains, model)
ev.add_chains(chains_infer)
ln_inv_evidence = ev.ln_evidence_inv


# Clip ln_inv_evidence to prevent inf
if np.isinf(ln_inv_evidence) or np.isnan(ln_inv_evidence):
    print("Warning: ln_inv_evidence is inf or nan, clipping to large value")
    ln_inv_evidence = np.clip(ln_inv_evidence, -1e10, 1e10)

# Compute errors and handle inf/nan
err_ln_inv_evidence = ev.compute_ln_inv_evidence_errors()
print(f"ln_inv_evidence: {ln_inv_evidence:.4f}")
print(f"err_ln_inv_evidence: {err_ln_inv_evidence}")

# Convert to log-evidence (ln(Z) = -ln(1/Z)) and extract first error
harmonic_logZ = -ln_inv_evidence
err_ln_evidence = err_ln_inv_evidence[0] if not (np.isinf(err_ln_inv_evidence[0]) or np.isnan(err_ln_inv_evidence[0])) else 1.0  # Default to 1.0 if inf/nan


# Print the computed log-evidence and its error in the requested format
print(f"Harmonic Log Evidence (RQSplineModel): {harmonic_logZ:.4f} +/- {err_ln_evidence:.4f}")

# Parse /data-synthetic/stats.dat to get Global Log-Evidence from synthetic
print(f"PyMultiNest log evidence (ln(Z)): {mn_logZ:.4f} +/- {err_mn_logZ:.4f}")


### Question

Why does the Harmonic evidence estimator require different inputs for `ln_posterior` when the prior is uniform, depending on whether the sampling is performed from the posterior distribution (e.g., via PyMultiNest) or from a uniform scan over the prior volume?

In other words, if the prior is uniform in both cases, why must I pass `log_weights` (posterior weights) in the posterior-sampled case, but `log_likelihoods` in the prior-sampled case, to obtain consistent evidence estimates with Harmonic?

---

### Summary of the answer

The key difference lies in the **sampling distribution of the data**:

- When samples are drawn **from the posterior** (Case A), the provided `log_weights` already encode the posterior density proportional to ℒ(θ) × π(θ). Thus, passing `ln_posterior = log_weights` correctly informs Harmonic about the distribution of samples and yields accurate evidence estimates.

- When samples are drawn **from the prior** uniformly (Case B), all posterior weights are equal (e.g., 1), which does not represent the posterior distribution. In this case, you must pass `ln_posterior = log_likelihoods` so that Harmonic knows the posterior is proportional to the likelihood over the uniform prior samples. This allows Harmonic to correctly compute the evidence by integrating the likelihood over the prior volume.

Therefore, even with a uniform prior in both cases, the difference in sampling distribution — posterior vs prior — dictates whether `ln_posterior` should be set to posterior weights or likelihood values for the evidence estimation to be consistent and accurate.
